In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
import nltk
import  sklearn
from collections import Counter
stemmer = SnowballStemmer('english')
news = pd.read_csv('/home/linu/news_articles.csv')
news.head()
tokenizer = ToktokTokenizer()
stops = set(stopwords.words('english'))
from sklearn.decomposition import LatentDirichletAllocation

no_of_recommends = 5
n_topics = 8

articletime_second = 120
articletime_seventh = 60
articletime_hundredth = 30

news = news[['Article_Id','Title','Content']].dropna()
contents = news["Content"].tolist()
title = news['Title']
article_id = news['Article_Id']

def clean_tokenize(document):
    document = re.sub('[^\w_\s-]',' ',document)
    tokens  = nltk.word_tokenize(document)
    cleaned_article = ' '.join([stemmer.stem(item) for item in tokens])   #stemming the tokenized corpus
    return cleaned_article

cleaned_articles = list(map(clean_tokenize,contents))

In [2]:
article_vocab = { }

article_vocab = enumerate(cleaned_articles)

total_words = []

for i in range(0, len(cleaned_articles)):
    tokens = nltk.word_tokenize(cleaned_articles[i])

    for w in tokens:
        total_words.append(w)
counts = Counter(total_words)

vocab = {j:i for i,j in enumerate(counts.keys())}

stops_removed = [i for i in vocab.keys() if i not in stops]

final_vocab = {j:i for i,j in enumerate(stops_removed)}


tf_idf = TfidfVectorizer(vocabulary=final_vocab,min_df=1)

article_vocabulary = tf_idf.fit_transform(cleaned_articles)

lda = LatentDirichletAllocation(n_components=n_topics,max_iter=1,random_state=0)

Lda_articlemat = lda.fit_transform(article_vocabulary)

wordtokens_article = [word.split() for word in cleaned_articles]   #we tokenize each word in our article to divide by word per second

In [3]:
print(len(wordtokens_article[0]),len(wordtokens_article[2]),len(wordtokens_article[3]))

222 227 325


In [4]:
# a function to create user profiles

def user_profiler(wordtokens,article_read,article_time):
    user_profile = []
    wordPer_second = 5
    #print(len(wordtokens_article))

    for i in range(len(wordtokens)):                                        

        average_time = (len(wordtokens[i])/wordPer_second) #length of wordtokslist by wps gives us avg time to read the article
         
        user_interest_timevalue = article_time[i]/average_time  #article_times divide by avg times of each article                   
       
        user_profile_generate = (article_read[i]*user_interest_timevalue)          #Ldamatrix[] * user_interest_time calculated                 
        
        user_profile.append(user_profile_generate)                                          

    return sum(user_profile)


userProfile_One = user_profiler([wordtokens_article[2],wordtokens_article[7],wordtokens_article[100]],
                         [Lda_articlemat[2],Lda_articlemat[7],Lda_articlemat[100]],
                         [120,60,30])

userProfile_Two = user_profiler([wordtokens_article[9],wordtokens_article[500],wordtokens_article[300]],
                         [Lda_articlemat[9],Lda_articlemat[500],Lda_articlemat[300]],
                         [111,120,180])

userProfile_Three = user_profiler([wordtokens_article[3],wordtokens_article[502],wordtokens_article[390]],
                           [Lda_articlemat[3],Lda_articlemat[502],Lda_articlemat[390]],
                           [200,120,100])


userprofile_List = [userProfile_One,userProfile_Two,userProfile_Three]
print(userProfile_One)

[0.07380161 0.07372557 1.20671618 0.0737193  1.15962764 0.07373607
 0.07371179 2.42932008]


In [12]:
def normalizer(norm) :
    n = []

    for profiles in userprofile_List:

        user_preffered_articles = cosine_similarity(profiles.reshape(1,-1),Lda_articlemat)
        a = np.argsort(user_preffered_articles).flatten()[::-1][:5]

        n.append(a)

    return n

similarityscore = normalizer(userprofile_List)

In [13]:
similarityscore

[array([2794, 1440, 1118, 3118,  140]),
 array([  88,  414, 2359, 1802, 1446]),
 array([3424, 4020, 4584, 3095, 4817])]

In [93]:
#news['Title'][similarityscore[2]]

In [115]:
for i in similarityscore:
    print('\n')
    print('Recommended Articles :')
    print('\n')
    print(news['Title'][i])



Recommended Articles :


2794    J K  8  Including 4 CRPF Jawans  Injured in An...
1440     Pokemon Go  guide  List of every Pokemon with...
1118    India replaces the US as world s second-bigges...
3118    Nagaland  Mob Drags  Rapist  Out of Dimapur Ja...
140     Nivin Pauly s Premam chosen as Apple Music s B...
Name: Title, dtype: object


Recommended Articles :


88       Vijay s Theri to clash with Suriya s 24 in April
414     Overseas box office collection   Kanche  conti...
2359    Donald Trump is the latest villain M O D A A K...
1802    Complete 2016 Badminton Schedule  Men s and Wo...
1446    Micromax Yu Yuphoria  6 Reasons Why it May Rep...
Name: Title, dtype: object


Recommended Articles :


3424    Delhi police bust Jaish-e-Mohammad terror cell...
4020    Reliance to launch online fashion shopping por...
4584    Lionel Messi is global brand ambassador of Tat...
3095    Bali Nine Case  Indonesia President Widodo Off...
4817    Six Essential Security Tips Every Whatsapp Us

In [119]:
np.argmax(Lda_articlemat[0])

2